In [33]:
import yaml
import os
import logging as log
import pprint

In [34]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Bar(y=[2, 3, 1]))
fig.show()

In [151]:
# Parse experiment yaml file
experiments_path="./experiments/regression_test.yaml"

# Get experiment information from yaml file.
experiment_params = yaml.load(open(experiments_path))

regression_tests_dir = os.path.expandvars(experiment_params['regression_tests_dir'])
params_dir = os.path.expandvars(experiment_params['params_dir'])
dataset_dir = os.path.expandvars(experiment_params['dataset_dir'])
executable_path = os.path.expandvars(experiment_params['executable_path'])

datasets_to_run = experiment_params['datasets_to_run']
regression_params = experiment_params['regression_parameters']

# Build dictionary from parameter name to list of parameter values
param_name_to_values = dict()
for regression_param in regression_params:
    param_name_to_values[regression_param['name']] = regression_param['values']

[{'values': [2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4], 'name': 'smartNoiseSigma'}]


In [152]:
# Retrieve stats, if they are not there, try to collect them:
full_stats_path = os.path.join(regression_tests_dir, "all_stats.yaml")
stats = dict()
if os.path.isfile(full_stats_path):
    log.info("Found existent stats. Opening full stats from:" + full_stats_path)
    stats = yaml.load(open(full_stats_path))
else:
    log.info("Collecting full stats.")
    # Collect all yaml results for a given parameter name:
    for regression_param in regression_params:
        # Redirect to param_name_value dir param_name = regression_param['name']
        param_name = regression_param['name']
        stats[param_name] = dict()
        for param_value in regression_param['values']:
            results_dir = os.path.join(regression_tests_dir, param_name, str(param_value))
            # Redirect to modified params_dir
            params_dir = os.path.join(results_dir, 'params')
            stats[param_name][param_value] = dict()
            for dataset in datasets_to_run:
                dataset_name = dataset['name']
                pipelines_to_run = dataset['pipelines']
                stats[param_name][param_value][dataset_name] = dict()
                for pipeline in pipelines_to_run:
                    results_file = os.path.join(results_dir, dataset_name, pipeline, "results.yaml")
                    if os.path.isfile(results_file):
                        stats[param_name][param_value][dataset_name][pipeline] = yaml.load(open(results_file,'r'))
                    else:
                        log.warning("Could not find results file: {}. Adding cross to boxplot...".format(results_file))
                        stats[param_name][param_value][dataset_name][pipeline] = False

    # Save all stats in regression tests root directory for future usage.
    with open(full_stats_path, 'w') as outfile:
        outfile.write(yaml.dump(stats))
    
    # Push to the cloud?!

I0727 13:07:19.086276 15219 <ipython-input-152-5c1f4edd58c2>:5] Found existent stats. Opening full stats from:/home/tonirv/Code/spark_vio_evaluation/regression_tests/all_stats.yaml


In [37]:
# Display plots for that result
# pprint.pprint(stats)

In [253]:
import plotly.graph_objects as go

def plot(x_data, y_data):
    colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)',
              'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)']

    fig = go.Figure()

    for xd, yd in zip(x_data, y_data):
            fig.add_trace(go.Box(
                y=yd,
                name=xd,
                boxpoints='all',
                jitter=0.5,
                whiskerwidth=0.2,
                marker_size=2,
                line_width=1)
            )

    fig.update_layout(
        title='Parameter: ' + param_name + ', dataset: ' + dataset_name,
        yaxis=dict(
            autorange=True,
            showgrid=True,
            zeroline=True,
            dtick=5,
            gridcolor='rgb(255, 255, 255)',
            gridwidth=1,
            zerolinecolor='rgb(255, 255, 255)',
            zerolinewidth=2,
        ),
        margin=dict(
            l=40,
            r=30,
            b=80,
            t=100,
        ),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        showlegend=False
    )

    fig.show()
    
def plot2():
    colors = ['#3D9970',  '#FF4136', '#FF851B']
    
    names = ['S', 'SP', 'SPR']
    
    x_data = [1, 1, 1, 1, 1, 1,
              2, 2, 2, 2, 2, 2]
    
    y_data= [[0.2, 0.2, 0.6, 1.0, 0.5, 0.4, 0.2, 0.7, 0.9, 0.1, 0.5, 0.3],
            [0.6, 0.7, 0.3, 0.6, 0.0, 0.5, 0.7, 0.9, 0.5, 0.8, 0.7, 0.2],
            [0.1, 0.3, 0.1, 0.9, 0.6, 0.6, 0.9, 1.0, 0.3, 0.6, 0.8, 0.5]]

    fig = go.Figure()
    for i in xrange(len(names)):
        fig.add_trace(go.Box(
            y=y_data[i],
            x=x_data,
            name=names[i],
            marker_color=colors[i]
        ))

    fig.update_layout(
        yaxis_title='ATE errors',
        boxmode='group' # group together boxes of the different traces for each value of x
    )
    fig.show()

In [254]:
plot2()


In [148]:
# Organize stats as for Dataset, Param Name, Param values (x_data)
# TODO use tidy panda... for storing errors

# for each dataset and param_name -> one plot
# for each param_value -> one x_axis
# for each pipeline -> one boxplot
# for each dataset, param_name, param_value and pipeline -> error
stats_list = []
for param_name in stats:
    for param_value in stats[param_name]:
        for dataset_name in stats[param_name][param_value]:
            for pipeline in stats[param_name][param_value][dataset_name]:
                result = stats[param_name][param_value][dataset_name][pipeline]['absolute_errors'].np_arrays['error_array']
                stats_list.append([param_name, param_value, dataset_name, pipeline, result])

#plot(param_name, dataset_name, x_data, y_data)

In [266]:
df = pd.DataFrame.from_records(stats_list)
df.columns = ['Param Name', 'Param Value', 'Dataset Name', 'Pipe Type', 'ATE errors']
df.set_index(['Param Name', 'Dataset Name'], inplace = True)
#df = df['ATE errors'].apply(lambda x: pd.Series(x)).stack().reset_index(level=4, drop=True).to_frame('ATE errors')
#df.boxplot(column=['S'])
df

Param Value Pipe Type  \
Param Name      Dataset Name                          
smartNoiseSigma V1_01_easy            2.0         S   
                V1_01_easy            2.0       SPR   
                MH_01_easy            2.0         S   
                MH_01_easy            2.0       SPR   
                V1_01_easy            3.0         S   
                V1_01_easy            3.0       SPR   
                MH_01_easy            3.0         S   
                MH_01_easy            3.0       SPR   
                V1_01_easy            2.2         S   
                V1_01_easy            2.2       SPR   
                MH_01_easy            2.2         S   
                MH_01_easy            2.2       SPR   
                V1_01_easy            3.2         S   
                V1_01_easy            3.2       SPR   
                MH_01_easy            3.2         S   
                MH_01_easy            3.2       SPR   
                V1_01_easy            2.4         S   
                V1_01_easy            2.4       SPR   
                MH_01_easy            2.4         S   
                MH_01_easy            2.4       SPR   
                V1_01_easy            3.4         S   
                V1_01_easy            3.4       SPR   
                MH_01_easy            3.4         S   
                MH_01_easy            3.4       SPR   
                V1_01_easy            2.6         S   
                V1_01_easy            2.6       SPR   
                MH_01_easy            2.6         S   
                MH_01_easy            2.6       SPR   
                V1_01_easy            2.8         S   
                V1_01_easy            2.8       SPR   
                MH_01_easy            2.8         S   
                MH_01_easy            2.8       SPR   

                                                                     ATE errors  
Param Name      Dataset Name                                                     
smartNoiseSigma V1_01_easy    [0.10997670458005289, 0.10517572913663793, 0.1...  
                V1_01_easy    [0.13134755458566505, 0.1262207963699808, 0.12...  
                MH_01_easy    [0.2151444667711187, 0.2157468309814195, 0.216...  
                MH_01_easy    [0.27152230985699444, 0.2716027138288078, 0.27...  
                V1_01_easy    [0.10524378348464582, 0.10130184371967772, 0.0...  
                V1_01_easy    [0.142529469669964, 0.13837671696192647, 0.134...  
                MH_01_easy    [0.2621647134833297, 0.26193382036350643, 0.26...  
                MH_01_easy    [0.22163350964595124, 0.22152040800082398, 0.2...  
                V1_01_easy    [0.15221248678273705, 0.14793509054390908, 0.1...  
                V1_01_easy    [0.1256631606142888, 0.12203396205882884, 0.12...  
                MH_01_easy    [0.20966290130595294, 0.20999767462721666, 0.2...  
                MH_01_easy    [0.21952022263183807, 0.21981717337110646, 0.2...  
                V1_01_easy    [0.09785664059397901, 0.09267139762164138, 0.0...  
                V1_01_easy    [0.1310147413454226, 0.12539454567538869, 0.12...  
                MH_01_easy    [0.22771362099988496, 0.22756626438640226, 0.2...  
                MH_01_easy    [0.20944104965172064, 0.20926471912655772, 0.2...  
                V1_01_easy    [0.08319468731182557, 0.07767729165039826, 0.0...  
                V1_01_easy    [0.14199617467940545, 0.1361275000855932, 0.13...  
                MH_01_easy    [0.19496875915929193, 0.19512576901798426, 0.1...  
                MH_01_easy    [0.27401621349947813, 0.2742909420948625, 0.27...  
                V1_01_easy    [0.10878469304380378, 0.09976441568897954, 0.0...  
                V1_01_easy    [0.13352836019851466, 0.1244023215499162, 0.11...  
                MH_01_easy    [0.22442579681956074, 0.2248948126062282, 0.22...  
                MH_01_easy    [0.23109794714348147, 0.23156511402988203, 0.2...  
                V1_01_easy 

In [267]:
y_data = df.loc["smartNoiseSigma"].loc[2.0].loc['V1_01_easy'].loc['S']
x_data = [2.0]

KeyError: u'the label [2.0] is not in the [index]'

In [223]:
plot(x_data, y_data)

In [307]:

import plotly.express as px
tips = px.data.tips()
fig = px.box(tips, y="total_bill")
fig.show()
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2.00,Male,No,Sun,Dinner,2
7,26.88,3.12,Male,No,Sun,Dinner,4
8,15.04,1.96,Male,No,Sun,Dinner,2
9,14.78,3.23,Male,No,Sun,Dinner,2


In [349]:
import plotly.express as px
tidy = df.loc["smartNoiseSigma"].loc['V1_01_easy']
tidy.set_index(['Param Value', 'Pipe Type'], inplace = True)


In [350]:
tidy_2 = tidy['ATE errors'].apply(lambda x: pd.Series(x)).stack().reset_index(level=2, drop=True).to_frame('ATE errors')
tidy_2.reset_index(level=['Pipe Type', 'Param Value'], drop=False, inplace=True)

In [351]:
fig = px.box(tidy_2, x='Param Value', y="ATE errors", points="all", color="Pipe Type")
fig.show()